# Demonstrating Keras saving feature
## Reusing models others (or you!) have trained
------
Code used: [Save and load](https://www.tensorflow.org/tutorials/keras/save_and_load)

## Step 0: Imports and initialization

We'll use the same dataset and model from the classify_digits notebook, also found in this directory.

In [3]:
import os

import tensorflow as tf
from tensorflow import keras

In [54]:
# This function will create a completely new model with the same structure 
# (useful for determining if loading is sucessful)
def create_new_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)), 
        keras.layers.Dense(128, activation='relu'), 
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    return model

In [4]:
# Creating a model (see the classify_digits notebook)
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images/255.0
test_images = test_images/255.0
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)), 
    keras.layers.Dense(128, activation='relu'), 
    keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 1: Saving

### Checkpoints

Checkpoints let you resume training. They
* are created with the `ModelCheckpoint` callback at the end of each training epoch
* usually use the `.ckpt` file extension
* have their filenames appended with indexing data when they are created while training
* don't usually include model data (unless `save_weights_only=False`) so they're smaller than entire models and useful for training the same model repeatedly

In [53]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/checkpoint.ckpt', # Gets appened with indexing data later
    save_weights_only=True) # Smaller checkpoint size, otherwise saves whole model

model.fit(train_images, 
          train_labels,  
          epochs=5,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 51us/sample - loss: 0.1156 - accuracy: 0.9562 - val_loss: 0.4563 - val_accuracy: 0.88700 - ETA: 0s - loss: 0.1157 - accuracy: 
Epoch 2/5
60000/60000 [==============================] - 3s 51us/sample - loss: 0.1122 - accuracy: 0.9585 - val_loss: 0.4549 - val_accuracy: 0.8864
Epoch 3/5
60000/60000 [==============================] - 3s 53us/sample - loss: 0.1101 - accuracy: 0.9590 - val_loss: 0.4459 - val_accuracy: 0.8894
Epoch 4/5
60000/60000 [==============================] - 3s 52us/sample - loss: 0.1093 - accuracy: 0.9590 - val_loss: 0.4782 - val_accuracy: 0.8847
Epoch 5/5
60000/60000 [==============================] - 3s 51us/sample - loss: 0.1076 - accuracy: 0.9602 - val_loss: 0.4650 - val_accuracy: 0.8857


### Models

Unlike checkpoints, models include **full information** about both weights and structure. This makes them useful outside of saving your own work because they do not require users create their own models with the same architecture.     
Tensorflow by default can export into 2 types of files:
1. `SavedModel` - This format is best used within Tensorflow applications. However, it is newer (there may be compatibility issues with older versions of TF) and not as widely adopted by other libraries.    
2. `HDF5` - This format is standardized and guaranteed to work with older versions of Tensorflow.

In [48]:
# Python runs into errors if a cross-platform path isn't created
model.save(os.path.join('models', 'example_model'))
# Ending the filepath with .h5 extension will create a HDF5 file
model.save(os.path.join('models', 'example_model')) 
# Calling model.save_weights(filepath) will save weights only (which is almost equivalent to a checkpoint)

INFO:tensorflow:Assets written to: models\example_model\assets
INFO:tensorflow:Assets written to: models\example_model\assets


## 2: Loading 

### Checkpoints

Lets load the checkpoint we created above. Note that the new model we use must be the **exact same structure** as the checkpoint's.

In [55]:
new_model = create_new_model()
ckpt = tf.train.Checkpoint(net=new_model)
# When dealing with more than one checkpoint, consider using a CheckpointManager
# ckpt_manager = tf.train.CheckpointManager(ckpt, 'checkpoints', max_to_keep = 1)

In [62]:
ckpt.restore('checkpoints/checkpoint.ckpt'); # Loads the checkpoint from path
model.evaluate(train_images,  train_labels, verbose=2)

60000/1 - 1s - loss: 0.1013 - accuracy: 0.9587


[0.10999159338002404, 0.9586833]

### Models

Models are easy to load: simply call `keras.models.load_model` and the appropriate weights and nodes will be loaded! Like the saving models, it takes both `SavedModel` and `HDF5` formats.

In [61]:
loaded_model = keras.models.load_model(os.path.join('models', 'example_model'))
loaded_model_h5 = keras.models.load_model(os.path.join('models', 'example_model.h5'))
print('\nOriginal model\n' + '-'*10)
model.evaluate(train_images,  train_labels, verbose=2)
model.evaluate(test_images,  test_labels, verbose=2)
print('\nLoaded model\n' + '-'*10)
loaded_model.evaluate(train_images,  train_labels, verbose=2)
loaded_model.evaluate(test_images,  test_labels, verbose=2)
print('\nLoaded model (HDF5) \n' + '-'*10)
loaded_model_h5.evaluate(train_images,  train_labels, verbose=2);
loaded_model_h5.evaluate(test_images,  test_labels, verbose=2);


Original model
----------
60000/1 - 2s - loss: 0.1013 - accuracy: 0.9587
10000/1 - 0s - loss: 0.2362 - accuracy: 0.8857

Loaded model
----------
60000/1 - 2s - loss: 0.1392 - accuracy: 0.9516
10000/1 - 0s - loss: 0.2444 - accuracy: 0.8806

Loaded model (HDF5) 
----------
60000/1 - 2s - loss: 0.1392 - accuracy: 0.9516
10000/1 - 0s - loss: 0.2444 - accuracy: 0.8806


Expectedly, the loaded models yield exactly the same results as the original model.

## Footnotes:
* In some documentation, the author refers to a class called `Saver`. This has been removed and its functionality ported to `Checkpoint` in Tensorflow 2.0+.